# Simple SIR/SEIR simulation for France (without deaths)
---

In [90]:
from __future__ import division # always-float division
import numpy as np
import pandas as pd
import glob
import pprint
import os
import requests
from datetime import date

# Easy interactive plots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# Interactive plots in notebook
from IPython.display import HTML, Image, display
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout, ToggleButton, ToggleButtons, fixed, Checkbox

# Maths
from scipy.integrate import odeint
import scipy.signal.windows as window
from sklearn.preprocessing import normalize
import scipy.stats as stats

#Import from utils
from utils import *


In [91]:
# Relative path to EPI data folder
DATA_PATH = './data/clean/EPI'
# Download OWID & Santé Publique data, write new file if new date
existing = glob.glob(DATA_PATH+"/*"+str(date.today())+".csv")

In [92]:
owid_file = update_owid(DATA_PATH)

Downloaded Our World In Data Coronavirus data to 
	./data/clean/EPI/owid_2021-04-14.csv


### Fonctions

In [93]:
def country_covid(country, owid_file):
    """
    Extracts 'country' time series from OWID database saved in 'owid_file'. Can add hospital data if 'country'==France and 'hosp_file' is specified.
    Time series starts when infected are positive
    """
    # Get ISO-3 code for country to later use as index for OWID database
    try:
        code = pycountry.countries.search_fuzzy(country)[0].alpha_3
    except LookupError:
        print(f'{country} not found in country dictionary.')
        return
    
    covid_data, country_attrs = extract_owid(owid_file, country_code=code)
    
    covid_data = covid_data.sort_values(by='date')
    covid_data = covid_data.reset_index(drop=True)
    # Oldest EPI values are all 0 (I, R, D)
    covid_data.loc[0, covid_data.columns != 'date'] = covid_data.loc[0, covid_data.columns != 'date'].apply(lambda x: 0)
    # Forward-fill NaN: old value is maintained until not-NaN value
    covid_data.ffill(axis=0, inplace=True)
    # Rename columns
    covid_data.columns = ['date', 'I', 'D', 's']
    # Compute S
    #covid_data['S'] = country_attrs['population'] - covid_data['I'] - covid_data['D']
    covid_data['S'] = country_attrs['population'] - covid_data['I'] 
    covid_data = covid_data[covid_data['I'] > 0]
    covid_data.reset_index(drop=True, inplace=True)
    covid_data['N_effective'] = country_attrs['population'] - covid_data['D']
    covid_data.bfill(axis=0, inplace=True)

    return covid_data, country_attrs

In [94]:
def get_data(country):
    
    EPI_data, country_attrs = country_covid(country, owid_file)
    df = EPI_data.drop(['N_effective'], axis=1).reindex(columns=['date', 'S', 'I', 'R','D', 's'])
    df.ffill(axis=0, inplace=True)
    df.bfill(axis=0, inplace=True)
    df.R.fillna(0, inplace=True)
    
    #Return an array of cumulated infected cases
    return df.loc[0:107, ['I']].values.ravel()

## Model and Interactive plot

In [95]:
import ipywidgets as widgets
from ipywidgets import interact

In [96]:
%matplotlib inline
#%matplotlib widget

In [97]:
france_data = get_data('France')

In [98]:
"""Main function"""
def simulation_SIR(beta, gamma, sigma=None, SEIR=False):
    
    def ODE(start_values, t, beta, gamma, sigma = None, SEIR=False):
    
        if SEIR:
            S, E, I, R = start_values
            N = S + E + I + R
            
            #Compartment derivatives
            dSdt = -beta*I*S/N
            dEdt = beta*I*S/N - sigma*E
            dIdt = sigma*E - gamma*I
            dRdt = gamma*I
            return [dSdt, dEdt, dIdt, dRdt]
        
        elif not SEIR:
            S, I, R = start_values
            N = S + I + R

            #Compartment derivatives
            dSdt = -beta*I*S/N
            dIdt = beta*I*S/N - gamma*I
            dRdt = gamma*I
            return [dSdt, dIdt, dRdt]

    #Initialization
    #Init number of days
    nb_days = 108
    
    #Init population (subset of total French population)
    pop = country_attrs['population']/350
    
    if SEIR:
        initE, initI, initR, initN = [1.0, 3.0, 0.0, pop]
        # initial Susceptible
        initS = initN - (initE + initI + initR)

        S_final = [initS]
        E_final = [initE]
        I_final = [initI]
        R_final = [initR]

        inputs = (initS, initE, initI, initR)
        
    elif not SEIR:
        initI, initR, initN = [3.0, 0.0, pop]
        # initial Susceptible
        initS = initN - (initI + initR)

        S_final = [initS]
        I_final = [initI]
        R_final = [initR]

        inputs = (initS, initI, initR)
    
    #Init cumulative cases
    cases_cumul = [initI]
    
    #Timeline
    tspan = np.arange(1, nb_days+1, 1)
    
    #Solve equa diff system
    if SEIR:
        res = odeint(ODE, [initS, initE, initI, initR], tspan, args=(beta, gamma, sigma, SEIR))

        S_final, E_final, I_final, R_final = res[:,0], res[:,1], res[:,2], res[:,3]

        #Compute difference 
        deriv_S = list(np.diff(S_final,1))
        deriv_E = list(np.diff(E_final,1))

        #Compute additional infected cases
        increment = deriv_E - np.abs(deriv_S)

        for i in range(nb_days-1):
            cases_cumul.append(cases_cumul[i] + np.abs(increment[i]))

    elif not SEIR:
        res = odeint(ODE, [initS, initI, initR], tspan, args=(beta, gamma, SEIR))

        S_final, I_final, R_final = res[:,0], res[:,1], res[:,2]

        #Compute difference 
        deriv_S = list(np.diff(S_final,1))

        #Compute additional infected cases
        for i in range(nb_days-1):
            cases_cumul.append(cases_cumul[i]+np.abs(deriv_S[i]))
        
    bbox = dict(boxstyle="round", fc="0.8")
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=tspan, y=S_final, mode='lines', name='S',
                 line = dict(color='royalblue', width=4), opacity=0.45))
    if SEIR:
        fig.add_trace(go.Scatter(x=tspan, y=E_final, mode='lines', name='E',
                     line = dict(color='purple', width=4), opacity=0.45))
    fig.add_trace(go.Scatter(x=tspan, y=I_final, mode='lines', name='I',
                 line = dict(color='red', width=4), opacity=0.45))
    fig.add_trace(go.Scatter(x=tspan, y=R_final, mode='lines', name='R', 
                             line = dict(color='green', width=4), opacity=0.45))
    fig.add_trace(go.Scatter(x=tspan, y=cases_cumul, mode='lines+markers', name='Cumulated infections',
                 line = dict(color='orange', width=3.5)))
    fig.add_trace(go.Scatter(x=tspan, y=france_data, mode='lines', name='FRA observed infections',
                 line = dict(color='darkorange', width=3.5, dash='dash'), opacity=0.8))
    
    fig.update_layout(title='Compartmental model',
                        title_font_family="Times New Roman",
                           xaxis_title='Days after first infected',
                           yaxis_title='Counts',
                           title_x=0.4,
                          width=950, height=600,
                     legend=dict(y=0.5, font_size=14),
                     font=dict(family="Courier New, monospace", size=15,color="black"))
    
    img_bytes = fig.to_image(format="png")
    display(Image(img_bytes))
    #fig.show()

In [99]:
interact(simulation_SIR, 
         #Input widgets
         beta=widgets.FloatSlider(min=0.0001, max=3.0, step=0.01, value=0.5), 
         gamma=widgets.FloatSlider(min=0.0001, max=1.5, step=0.01, value=0.1),
         sigma=widgets.FloatSlider(min=0.0001, max=1.0, step=0.01, value=0.1),
         SEIR=True)

interactive(children=(FloatSlider(value=0.5, description='beta', max=3.0, min=0.0001, step=0.01), FloatSlider(…

<function __main__.simulation_SIR(beta, gamma, sigma=None, SEIR=False)>